You need fasta file and tar.gz file. See below.

In [1]:
!tree ../data/amp -hDL 3

[ 160 Sep  5 14:17]  ../data/amp
├── [216K Sep  5 15:44]  all_data.fa
└── [ 128 Sep  5 14:17]  prose
    └── [  96 Sep  5 14:17]  all_data
        └── [126K Sep  4 14:39]  amp_all_dlm_avg

3 directories, 1 file


For now we will use the tool from esm to read fasta file.  
Later I might find another way.   
To use it, you have to install esm:  
`pip install fair-esm`

Then, import esm as I did below.

you will need to extract the tar.gz file.

Use this [link](https://logfetch.com/wsl2-zip-unzip-files/) to unzip and untar the file.

You may try with windows tool also. Ping me if you will need any help.
   
Code below I adjusted form this notebook  https://github.com/facebookresearch/esm/blob/main/examples/sup_variant_prediction.ipynb    
    
It will create X as an array and y as a list.

From them you should be able to create a dataframe.

In [ ]:
import esm
import torch
import numpy as np
np.random.seed(7)  # for reproducibility

: 

: 

In [3]:
path_fa = '../data/amp/all_data.fa'
# path_h5 = '../data/amp/prose/all_data/amp_all_dlm_avg.h5'
path_pt = '../data/amp/prose/all_data/amp_all_dlm_avg'
emb_layer = 'layer'
pool = 'avg'

In [4]:
ys = []
Xs = []
seq_id = []

for header, _seq in esm.data.read_fasta(path_fa):
    label = header.split('|')[-1]
    ys.append(float(label))   # see if you need integer or afloat
    seq = header.split('|')[-2]
    seq_id.append(seq)
    fn = f'{path_pt}/{header[1:]}.pt'
    embs = torch.load(fn)
    Xs.append(embs[f'{pool}_representations'][emb_layer])
Xs = torch.stack(Xs, dim=0).numpy()
print(len(ys))
print(Xs.shape)
print(len(seq_id))

4042
(4042, 6165)
4042


In [5]:
print(type(ys))
print(type(Xs))
print(type(seq_id))

<class 'list'>
<class 'numpy.ndarray'>
<class 'list'>


In [6]:
Xs

array([[ 0.1904762 ,  0.        ,  0.        , ..., -0.07591449,
        -0.01412157,  0.01358508],
       [ 0.        ,  0.02325581,  0.04651163, ...,  0.05043951,
        -0.0538577 , -0.12193269],
       [ 0.20000002,  0.        ,  0.        , ..., -0.08004365,
         0.07211193, -0.03114778],
       ...,
       [ 0.11320755,  0.01886792,  0.05660377, ..., -0.04887858,
        -0.03788646,  0.02409669],
       [ 0.        ,  0.        ,  0.        , ...,  0.01287932,
         0.0615775 ,  0.17003648],
       [ 0.        ,  0.        ,  0.04      , ..., -0.09218226,
         0.05127459,  0.05520697]], dtype=float32)

In [7]:
import pandas as pd
df = pd.DataFrame(Xs, index=seq_id ).reset_index().rename(columns={'index':'sequence'})
df

,sequence,0,1,2,3,4,5,6,7,8,...,6155,6156,6157,6158,6159,6160,6161,6162,6163,6164
0,AP02484,0.190476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190476,0.000000,...,-0.074216,0.125029,0.000287,0.031710,0.045591,-0.068033,0.031241,-0.075914,-0.014122,0.013585
1,AP02630,0.000000,0.023256,0.046512,0.023256,0.162791,0.046512,0.000000,0.093023,0.046512,...,-0.011951,-0.139132,-0.051193,0.108947,-0.014495,-0.107359,0.016725,0.050440,-0.053858,-0.121933
2,AP01427,0.200000,0.000000,0.000000,0.033333,0.066667,0.033333,0.066667,0.066667,0.000000,...,-0.034706,0.061255,-0.004903,0.041961,0.052808,-0.083791,0.009830,-0.080044,0.072112,-0.031148
3,AP02983,0.095238,0.190476,0.000000,0.000000,0.000000,0.047619,0.047619,0.142857,0.047619,...,-0.061731,0.040203,-0.007535,0.038144,0.009323,-0.134387,0.005756,-0.134904,-0.035848,0.051515
4,AP01815,0.041667,0.145833,0.000000,0.041667,0.125000,0.020833,0.000000,0.125000,0.000000,...,-0.013842,-0.147841,-0.013836,0.051293,0.002145,-0.141400,0.038845,-0.136291,0.054276,-0.005753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4037,UniRef50_C5DJ44,0.116279,0.023256,0.058140,0.011628,0.011628,0.093023,0.127907,0.046512,0.023256,...,0.004283,-0.087141,-0.016207,0.027803,-0.015408,-0.128817,-0.115971,-0.076693,0.077499,0.173814
4038,UniRef50_Q9XUP3,0.081395,0.023256,0.034884,0.023256,0.023256,0.023256,0.069767,0.069767,0.011628,...,-0.015495,-0.062792,0.001243,0.043942,0.015509,-0.140289,0.023302,-0.101669,0.015073,0.017768
4039,UniRef50_Q9Y573,0.113208,0.018868,0.056604,0.037736,0.037736,0.018868,0.113208,0.056604,0.018868,...,-0.006378,-0.136874,-0.010065,0.022892,0.006662,-0.144792,0.014618,-0.048879,-0.037886,0.024097
4040,UniRef50_Q54H44,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.066667,0.000000,0.133333,...,-0.002670,-0.064352,-0.036118,0.038847,-0.005325,-0.162470,-0.013375,0.012879,0.061577,0.170036


In [8]:
df = pd.concat([df, pd.DataFrame(ys, columns =['label'])], axis=1)
df

,sequence,0,1,2,3,4,5,6,7,8,...,6156,6157,6158,6159,6160,6161,6162,6163,6164,label
0,AP02484,0.190476,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190476,0.000000,...,0.125029,0.000287,0.031710,0.045591,-0.068033,0.031241,-0.075914,-0.014122,0.013585,1.0
1,AP02630,0.000000,0.023256,0.046512,0.023256,0.162791,0.046512,0.000000,0.093023,0.046512,...,-0.139132,-0.051193,0.108947,-0.014495,-0.107359,0.016725,0.050440,-0.053858,-0.121933,1.0
2,AP01427,0.200000,0.000000,0.000000,0.033333,0.066667,0.033333,0.066667,0.066667,0.000000,...,0.061255,-0.004903,0.041961,0.052808,-0.083791,0.009830,-0.080044,0.072112,-0.031148,1.0
3,AP02983,0.095238,0.190476,0.000000,0.000000,0.000000,0.047619,0.047619,0.142857,0.047619,...,0.040203,-0.007535,0.038144,0.009323,-0.134387,0.005756,-0.134904,-0.035848,0.051515,1.0
4,AP01815,0.041667,0.145833,0.000000,0.041667,0.125000,0.020833,0.000000,0.125000,0.000000,...,-0.147841,-0.013836,0.051293,0.002145,-0.141400,0.038845,-0.136291,0.054276,-0.005753,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4037,UniRef50_C5DJ44,0.116279,0.023256,0.058140,0.011628,0.011628,0.093023,0.127907,0.046512,0.023256,...,-0.087141,-0.016207,0.027803,-0.015408,-0.128817,-0.115971,-0.076693,0.077499,0.173814,0.0
4038,UniRef50_Q9XUP3,0.081395,0.023256,0.034884,0.023256,0.023256,0.023256,0.069767,0.069767,0.011628,...,-0.062792,0.001243,0.043942,0.015509,-0.140289,0.023302,-0.101669,0.015073,0.017768,0.0
4039,UniRef50_Q9Y573,0.113208,0.018868,0.056604,0.037736,0.037736,0.018868,0.113208,0.056604,0.018868,...,-0.136874,-0.010065,0.022892,0.006662,-0.144792,0.014618,-0.048879,-0.037886,0.024097,0.0
4040,UniRef50_Q54H44,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.066667,0.000000,0.133333,...,-0.064352,-0.036118,0.038847,-0.005325,-0.162470,-0.013375,0.012879,0.061577,0.170036,0.0


## Sequence Classification using Deep Learning in TensorFlow

In [11]:
df = df.sample(frac=1, random_state=42)
print(df.shape)
df.head(5)

(4042, 6167)


,sequence,0,1,2,3,4,5,6,7,8,...,6156,6157,6158,6159,6160,6161,6162,6163,6164,label
368,AP02518,0.105263,0.052632,0.052632,0.105263,0.105263,0.000000,0.052632,0.105263,0.000000,...,-0.129499,-0.008706,0.033810,-0.001945,-0.138693,0.015822,-0.107180,-0.038549,0.014884,1.0
2554,UniRef50_Q96N95,0.040000,0.000000,0.000000,0.000000,0.000000,0.120000,0.160000,0.000000,0.040000,...,-0.087970,-0.021962,0.045560,0.010681,-0.095740,0.054584,-0.048101,0.071182,0.045190,0.0
2308,UniRef50_Q08BB3,0.043478,0.086957,0.021739,0.043478,0.000000,0.043478,0.043478,0.152174,0.021739,...,-0.089049,-0.002336,0.034276,0.001333,-0.134315,0.009472,-0.055397,0.027155,0.024235,0.0
1959,AP02347,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.044095,0.006911,0.020959,-0.007336,-0.088146,0.026249,-0.077495,0.015968,0.089023,1.0
2542,UniRef50_Q3INV7,0.102804,0.074766,0.028037,0.074766,0.000000,0.009346,0.102804,0.093458,0.046729,...,-0.018032,0.007449,0.036262,0.003307,-0.188603,0.022977,-0.003067,0.117292,0.025237,0.0


In [1]:

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

NameError: name 'np' is not defined

In [ ]:
# Data preprocessing
y = df['label']
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)